# Grid World Game
Based on the tutorial [here](https://towardsdatascience.com/reinforcement-learning-implement-grid-world-from-scratch-c5963765ebff) by Jeremy Zhang

In [1]:
import numpy as np

# Global Variables
BOARD_ROWS = 3
BOARD_COLS = 4
WIN_STATE = (3, 0)
LOSE_STATE = (3, 1)
START = (0, 0)
WALLS = [(1, 1)]
DETERMINISTIC = True
LEARNING_RATE = 0.2
EXPLORATION_RATE = 0.3

## Rules
![The grid](GridWorldGrid.png)
 - The agent starts at the bottom-left corner and ends at either +1 or -1 (the reward). 
 - There are 4 possible actions (up, down, left, right).
 - If the agent hits the wall, it will remain in the same position.

Note that this first attempt is deterministic. 

## Board
For each iteration we need a way to return the reward given a particular state, take in the current state and action and return the next state, and a way to know when the current iteration is finished.

In [2]:
class Iteration:
    def __init__(self, state=START):
        self.board = np.zeros([BOARD_ROWS, BOARD_COLS])
        self.board[1,1] = -1
        self.state = state
        self.is_end = False

    def giveReward(self):
        if self.state == WIN_STATE:
            return 1
        elif self.state == LOSE_STATE:
            return -1
        else:
            return 0

    def next_position(self, action):
        """
        2 |
        1 |
        0 | 1 | 2 | 3
        """
        if DETERMINISTIC:
            if action == "up":
                next_state = (self.state[0], self.state[1] + 1)
            elif action == "down":
                next_state = (self.state[0], self.state[1] - 1)
            elif action == "left":
                next_state = (self.state[0] - 1, self.state[1])
            else:
                next_state = (self.state[0] + 1, self.state[1])
                
        # Check that the next state is legal
        if (next_state[0] >= 0) and (next_state[0] <= 3):
            if (next_state[1] >= 0) and (next_state[1] <= 2):
                if next_state not in WALLS:
                    self.state = next_state
        return self.state

    def is_end_func(self):
        if (self.state == WIN_STATE) or (self.state == LOSE_STATE):
            self.is_end = True

## Agent
### Value Iteration
The agent will learn a policy (a mapping from state to action), that instructs what the agent should do at each state.

Instead here we will use valie iteration to first learn a mapping of state to value (estimated reward), i.e. how much is each cell worth in reward? Based on the estimation at each state the agnet will choose the best action that gives the highest estimated reward.

Value iteration formula: $${\large V(S_t) \leftarrow V(S_t)+ \alpha \left[ V(S_{t+1}) - V(S_t) \right]}$$

As the name suggests, the value is updated at the end of each iteration, working backwards through the path that the agent took. When the agent first starts, all cells have value 0 (except the start/end). Once the agent reaches the end of the game, the reward propagates in a backwards fashion so the estimated value of all states along the way will be updated using the formula.

$V(S_t)$ on the left is the updated value of the state. on the right of the arrow $V(S_t)$ is the current value of the state, $ \alpha $ is the learning rate and $V(S_{t+1})$ is the value of the next state of the agent. The formula says that the updated value of a state is equal to the current value plus a temporal difference (what the agent learned from this iteration minus the previous estimate).

We need our agent to be able to 'play' the game. Once an iteration is finished, we need to be able to reset our agent ready for the next iteration. It would also be useful to have a helper method that displays the values at the end of each iteration.

### Exploration vs. Expoitation
Once the agent finds a path to get reward +1, should it stick to it foreger or give another path a chance in hope of finding a shorter one? This needs to be balanced to avoid the agent getting stuck in a local optimal. The action will be chosen based on a certain exploration rate.

In [3]:
class Agent:
    def __init__(self):
        self.states = []
        self.actions = ["up", "down", "left", "right"]
        self.iteration = Iteration()
        self.is_end = self.iteration.is_end
        self.learning_rate = LEARNING_RATE
        self.exploration_rate = EXPLORATION_RATE

        # Set up an initial empty set of rewards
        self.state_values = {}
        for i in range(BOARD_COLS):
            for j in range(BOARD_ROWS):
                self.state_values[(i,j)] = 0

    def take_action(self, action):
        position = self.iteration.next_position(action)
        return Iteration(state=position)

    def play(self, rounds = 10):
        i = 0
        while i < rounds:
            # to the end of the game back-propagate the reward
            if self.iteration.is_end:
                # back propagate
                reward = self.iteration.giveReward()
                # explicitly assign end state to reward values
                self.state_values[self.iteration.state] = reward
                print("Game end reward", reward)
                for s in reversed(self.states):
                    reward = self.state_values[s] + self.learning_rate * (reward - self.state_values[s])
                    self.state_values[s] = round(reward, 3)
                self.reset()
                i += 1
            else:
                action = self.choose_action()
                # append trace
                self.states.append(self.iteration.next_position(action))
                #print("current position {} action {}".format(self.iteration.state, action))
                # by taking the action, it reaches the next state
                self.iteration = self.take_action(action)
                # check whether we have reached the end
                self.iteration.is_end_func()
                #print("next state", self.iteration.state)
                #print("--------------------")

    def choose_action(self):
        # choose the action with the most expected value
        max_next_reward = 0
        action = ""
        if np.random.uniform(0,1) <= self.exploration_rate:
            action = np.random.choice(self.actions)
        else:
            # greedy action
            for a in self.actions:
                # if the action is deterministic
                next_reward = self.state_values[self.iteration.next_position(a)]
                if next_reward > max_next_reward:
                    action = a
                    max_next_reward = next_reward
        return action

    def reset(self):
        self.show_values()
        self.states = []
        self.iteration = Iteration()
        self.is_end = self.iteration.is_end

    def show_values(self):
        for i in range(0 , BOARD_ROWS):
            print("-------------------------")
            out = " | "
            for j in range(0, BOARD_COLS):
                out += str(self.state_values[(j,i)]) + " | "
            print(out)
        print("------------------------")

Finally, we can initialise our agent and play the game.

In [4]:
agent = Agent()
agent.play(50)

Game end reward 1
-------------------------
 | 0 | 0 | 0.2 | 1 | 
-------------------------
 | 0 | 0 | 0 | 0 | 
-------------------------
 | 0 | 0 | 0 | 0 | 
------------------------
Game end reward 1
-------------------------
 | 0.072 | 0 | 0.36 | 1 | 
-------------------------
 | 0 | 0 | 0 | 0 | 
-------------------------
 | 0 | 0 | 0 | 0 | 
------------------------
Game end reward 1
-------------------------
 | 0.078 | 0.08 | 0.488 | 1 | 
-------------------------
 | 0.015 | 0 | 0 | 0 | 
-------------------------
 | 0 | 0 | 0 | 0 | 
------------------------
Game end reward 1
-------------------------
 | 0.078 | 0.08 | 0.59 | 1 | 
-------------------------
 | 0.015 | 0 | 0 | 0 | 
-------------------------
 | 0 | 0 | 0 | 0 | 
------------------------
Game end reward 1
-------------------------
 | 0.197 | 0.08 | 0.672 | 1 | 
-------------------------
 | 0.051 | 0 | 0 | 0 | 
-------------------------
 | 0 | 0 | 0 | 0 | 
------------------------
Game end reward 1
------------------------